In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, sample_each, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [6]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(1000, 1035)

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 0 0 

## run

In [9]:
while epoch < config.n_epochs:
    time_start = time.time()
    sample_each(train_docs, topic_root, train=True)
    time_log = float(time.time() - time_start)
    
    sample_each(dev_docs, topic_root, train=False)
    sample_each(test_docs, topic_root, train=False)
    
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train=0
    ppl_dev = get_perplexity(dev_docs, topic_root)
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, ppl_dev, ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN:      VALID:       TEST: SPEC:             HIER:  \
        Time  Ep Ct    PPL         PPL         PPL     1     2     3 CHILD   
0   1.667183   0  0      0  523.243087  522.291448  0.06  0.29  0.50  0.87   
1   1.688059   1  0      0  519.177059  518.214660  0.06  0.33  0.51  0.88   
2   1.703319   2  0      0  513.110383  512.780887  0.06  0.36  0.50  0.89   
3   1.740450   3  0      0  512.158320  511.674471  0.06  0.36  0.50  0.91   
4   1.748168   4  0      0  511.302786  511.215596  0.06  0.36  0.50  0.91   
5   1.819271   5  0      0  509.776845  510.613602  0.06  0.36  0.50  0.90   
6   1.699607   6  0      0  511.487332  510.613602  0.06  0.36  0.50  0.90   
7   1.700828   7  0      0  508.402412  509.624214  0.06  0.36  0.50  0.90   
8   1.691052   8  0      0  508.309273  508.114282  0.06  0.36  0.50  0.91   
9   1.697628   9  0      0  511.102597  508.114282  0.06  0.35  0.50  0.91   
10  1.718008  10  0      0  511.871504  508.114282  0.06  0.37  0.50  0.91   
11  1.818515  11  0      0  511.252871  508.114282  0.06  0.37  0.50  0.90   
12  1.683208  12  0      0  513.015458  508.114282  0.06  0.37  0.51  0.89   
13  2.471273  13  0      0  510.230896  508.114282  0.06  0.38  0.49  0.89   
14  1.725868  14  0      0  511.471157  508.114282  0.06  0.45  0.51  0.95   
15  2.012767  15  0      0  510.802642  508.114282  0.06  0.45  0.50  0.95   
16  1.881742  16  0      0  512.466613  508.114282  0.06  0.44  0.51  0.95   
17  1.754559  17  0      0  511.337479  508.114282  0.06  0.45  0.51  0.95   
18  1.800101  18  0      0  512.400827  508.114282  0.06  0.45  0.51  0.95   
19  1.966097  19  0      0  511.723698  508.114282  0.06  0.45  0.51  0.95   
20  2.282601  20  0      0  509.236203  508.114282  0.06  0.46  0.50  0.96   
21  2.123910  21  0      0  508.928112  508.114282  0.06  0.45  0.51  0.95   
22  1.814540  22  0      0  509.759419  508.114282  0.06  0.45  0.51  0.95   
23  2.514012  23  0      0  511.536094  508.114282  0.06  0.37  0.50  0.91   
24  1.789379  24  0      0  510.776361  508.114282  0.06  0.38  0.50  0.91   
25  1.716793  25  0      0  510.614833  508.114282  0.06  0.45  0.51  0.95   
26  2.367953  26  0      0  510.188240  508.114282  0.06  0.37  0.51  0.89   
27  1.676732  27  0      0  512.798832  508.114282  0.06  0.36  0.50  0.90   
28  1.872300  28  0      0  511.666374  508.114282  0.06  0.36  0.49  0.90   
29  2.294336  29  0      0  508.799606  508.114282  0.06  0.35  0.51  0.89   
..       ...  .. ..    ...         ...         ...   ...   ...   ...   ...   
70  2.557405  70  0      0  510.838444  505.520414  0.06  0.45  0.51  0.95   
71  2.048017  71  0      0  509.868630  505.520414  0.06  0.45  0.51  0.95   
72  1.803219  72  0      0  509.993511  505.520414  0.06  0.46  0.51  0.95   
73  2.170357  73  0      0  508.886873  505.520414  0.06  0.46  0.51  0.95   
74  1.843200  74  0      0  508.520119  505.520414  0.06  0.39  0.49  0.91   
75  1.801988  75  0      0  508.988207  505.520414  0.06  0.46  0.51  0.95   
76  1.831269  76  0      0  508.067845  505.520414  0.06  0.45  0.51  0.95   
77  1.749460  77  0      0  505.630302  506.070398  0.06  0.39  0.50  0.91   
78  2.824527  78  0      0  510.225087  506.070398  0.06  0.38  0.50  0.90   
79  1.783543  79  0      0  511.151049  506.070398  0.06  0.44  0.51  0.95   
80  1.814502  80  0      0  510.297674  506.070398  0.06  0.45  0.51  0.95   
81  2.140080  81  0      0  509.547729  506.070398  0.06  0.46  0.51  0.95   
82  1.866834  82  0      0  509.531295  506.070398  0.06  0.38  0.51  0.91   
83  1.715529  83  0      0  510.812767  506.070398  0.06  0.38  0.50  0.91   
84  1.800321  84  0      0  511.011164  506.070398  0.06  0.38  0.50  0.90   
85  2.816565  85  0      0  511.263085  506.070398  0.06  0.37  0.50  0.90   
86  1.756937  86  0      0  510.782452  506.070398  0.06  0.38  0.48  0.91   
87  1.747971  87  0      0  508.437989  506.070398  0.06  0.45  0.51  0.95   
88  2.317929  88  0      0  508.008727  50

 0 1000 12646.0 ! nice love price bought sleeve cover ; pocket quality
   0-1 1000 4453.0 perfect top cover side zipper camera carry ! straps storage
     0-1-1 1000 798.0 put nice cord water dark works inside snug hard snaps
